In [20]:
import os
import json
from groq import Groq
from openai import OpenAI

In [21]:
#open dataset and store it as a JSON object
with open('../GSM-IC/GSM-IC_mstep.json', 'r') as file:
    questions = json.load(file)
with open('../GSM-IC/GSM-IC_2step.json', 'r') as file:
    questions2 = json.load(file)
    #initialize the Groq client. This allows us to query Llama3 (or another model) from my Groq account
#todo: store the api_key in a secure way!
client = Groq(
    # api_key=os.environ.get("GROQ_API_KEY")
    api_key='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'
)
# client = OpenAI(api_key=open("api.txt").read())

In [22]:
#todo: 1. answer_prompt is the current way of forcing the model to output its answer in predictable format. Find a better way to extract the numerical answer from the model's output. (maybe: require JSON output)

#todo: 2. clean up the answer. Ex. remove $, %, inches, etc. 

def single_query(query):
    """
    query the model with a single question
    input: what you would type in the chat box to ask the model a question
    output: the model's output in response to your query 
    """

    
    answer_prompt='The last sentence of your response must be in the form "The answer is [your answer]".'
    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": query + answer_prompt,
        }
    ],
    model="llama-3.1-70b-versatile",
    )
    response = chat_completion.choices[0].message.content
    return response 


In [23]:
#todo: shouldn't need to separate the two queries like this
def multiple_queries(question_type='original_question', num_questions=5):
    """
    query the model sequentially with multiple questions from the dataset 
    Args: question_type: specify whether to query the model with the original question or the new question 
        num_questions: the number of questions (mainly for testing a few questions at a time)
    
    """
    for idx, question in enumerate(questions):
        if idx < num_questions:
            query = question[question_type]
            response = single_query(query)
            print(response)
            print('--------------------------------------------------')

In [24]:
import tqdm
#todo: shouldn't need to separate the two queries like this
def get_answers(questions, question_type='original_question', num_questions=5):
    """
    query the model sequentially with multiple questions from the dataset 
    Args: question_type: specify whether to query the model with the original question or the new question 
        num_questions: the number of questions (mainly for testing a few questions at a time)
    
    """
    answers = {}
    for idx, question in tqdm.tqdm(list(enumerate(questions[:num_questions]))):
        if idx < num_questions:
            query = question[question_type]
            response = single_query(query)
            # print(response)
            try:
                answer = response.split("he answer is")[1]#.strip("$")
                # print(question['answer'], answer, question)
                answers[idx] = [question['answer'] in answer, answer]
            except Exception as e:
                print(e)
                # pass
                # print(response, question['answer'])
                # input()
        else:
            break
    return answers

In [ ]:
answers = get_answers(questions2, question_type='new_question', num_questions=500)

 19%|███████▉                                  | 94/500 [05:57<15:00,  2.22s/it]

In [ ]:
answers_steps = []
for steps in [3, 4, 5, 6, 7]:
    answers_steps.append(get_answers([i for i in questions if i['n_steps'] == steps], question_type='new_question', num_questions=500))

In [ ]:
x = [questions2]
for steps in [3, 4, 5, 6, 7]:
    x.append([i for i in questions if i['n_steps'] == steps])
y = [answers]
y.extend(answers_steps)

In [ ]:
import pickle
f=open("answers_llama_3_1_9b.obj", "wb")
pickle.dump(y, f)

In [ ]:
step_accs = [[0, 0] for _ in range(10)]
for q, a in zip(x, y):
    for k, v in a.items():
        question = q[k]
        step_accs[question['n_steps']][0] += int(v[0])
        step_accs[question['n_steps']][1] += 1

print(step_accs)
step_accs = [i[0] / i[1] for i in step_accs[2:8]]

import matplotlib.pyplot as plt

plt.ylabel("Accuracy")
plt.xlabel("Steps")
plt.title("Llama3-8b Accuracy")
plt.bar(list(range(2, 2+len(step_accs))), step_accs)